In [1]:
import re
from helpers import data

In [2]:
def parse(block):
    # Convert \n to spaces within block 
    passport = block.replace("\n", " ").split()
    return {s[:3]: s[4:] for s in passport}

In [3]:
passports = data(4, parser=parse, sep="\n\n")
passports[0]

{'byr': '1971',
 'ecl': 'hzl',
 'pid': '112040163',
 'eyr': '2023',
 'iyr': '2019',
 'hcl': '#b6652a',
 'hgt': '167cm'}

**Part 1:** Report number of passports with fields byr, iyr, eyr, hgt, hcl, ecl, pid (don't care about cid). 

In [4]:
required = { "byr", "iyr", "eyr", "hgt", "hcl", "ecl", "pid" }

In [5]:
# Attempt 1 
valid = 0 
for passport in passports:
    fields = set(passport.keys())
    valid += required.issubset(fields)

valid

192

In [6]:
# Attempt 2: Nevermind, way less readable
valid = 0 
for passport in passports: 
    fields = set(passport.keys())
    valid += fields | required == fields 
    
valid

192

In [7]:
# Norvig 
def parse_passport(passport: str) -> dict: 
    """Make a dict with all key: value entries in passport."""
    return dict(re.findall(r"([a-z]+):([^\s]+)", passport))

passports = data(4, parser=parse_passport, sep="\n\n")
sum(required.issubset(passport) for passport in passports)

192

**Part 2:** Validate field values now. 

In [8]:
in_between = lambda a, b: lambda x: a <= int(x) <= b

def in_between(a: int, b: int): 
    """HOF to check if x is in (a,b)."""
    def x_between(x: str) -> bool:
        return a <= int(x) <= b
    return x_between

def hgt(s: str) -> bool:
    if s[-2:] == "cm": 
        return in_between(150, 193)(s[:-2])
    elif s[-2:] == "in":
        return in_between(59, 76)(s[:-2])
    else: 
        return False
        
def hcl(s: str) -> bool:
    if s[0] != "#": 
        return False 
    if re.fullmatch("[0-9a-f]{6}$", s[1:]):
        return True 
    return False 


checks = {
    "byr": in_between(1920, 2002), 
    "iyr": in_between(2010, 2020), 
    "eyr": in_between(2020, 2030), 
    "hgt": hgt, 
    "hcl": hcl, 
    "ecl": lambda s: s in {"amb", "blu", "brn", "gry", "grn", "hzl", "oth"}, 
    "pid": lambda s: re.fullmatch("[0-9]{9}#", s)
}

# Cleaner method 
checks = dict(
    byr=in_between(1920, 2002), 
    iyr=in_between(2010, 2020), 
    eyr=in_between(2020, 2030), 
    hgt=hgt, 
    hcl=hcl, 
    ecl=lambda s: s in {"amb", "blu", "brn", "gry", "grn", "hzl", "oth"}, 
    pid=lambda s: re.fullmatch("[0-9]{9}$", s)
)

In [9]:
# Attempt 1
valid = 0 
for passport in passports:
    fields = set(passport.keys())
    # Fields should have all required 
    if not required.issubset(fields):
        continue 
    for field, func in checks.items():
        if not func(passport[field]):
            break 
    else:
        valid += 1

valid

101

In [10]:
# Norvig 
valid_passports = (all(field in passport 
                    and checks[field](passport[field])
                    for field in required
                   ) for passport in passports)
sum(valid_passports)

101